# Get IPs

In this notebook we explore how to get IPs from our IP list.

In [1]:
import requests

In [2]:
url = "https://freegeoip.app/csv/"

headers = {
    'accept': "application/csv",
    'content-type': "application/csv"
    }


response = requests.request("GET", url, headers=headers)

print(response.text)

24.17.221.217,US,United States,WA,Washington,Liberty Lake,99019,America/Los_Angeles,47.6517,-117.0838,881



In [3]:
url_ip = "https://freegeoip.app/csv/{}"

In [4]:
response = requests.request("GET", url_ip.format('23.116.185.114'))

In [5]:
print(response.text)

23.116.185.114,US,United States,IL,Illinois,Wheaton,60187,America/Chicago,41.8733,-88.1163,602



In [8]:
response.status_code

200

In [9]:
response2 = requests.request("GET", url_ip.format('::ffff:185.40.4.65'))

In [10]:
response2.text

'185.40.4.65,RU,Russia,,,,,Europe/Moscow,55.7386,37.6068,0\r\n'

In [76]:
response2.status_code == 200

True

That last one shows we need a way to add the attacker IP, and then the returned IP...

In [13]:
first_line = ','.join(('::ffff:185.40.4.65', '185.40.4.65,RU,Russia,,,,,Europe/Moscow,55.7386,37.6068,0\r\n'))

In [14]:
second_line = ','.join(('23.116.185.114', requests.request("GET", url_ip.format('23.116.185.114')).text))

In [15]:
second_line

'23.116.185.114,23.116.185.114,US,United States,IL,Illinois,Wheaton,60187,America/Chicago,41.8733,-88.1163,602\r\n'

In [16]:
lines = [first_line, second_line]

In [18]:
f = open('../data/ip_geos.csv', 'a')

In [19]:
f.writelines(lines)

In [20]:
f.close()

Ok going to write a class, GeolocateIPs, that will do this for me.

In [26]:
with open('../data/honeypot_10ksamp_attackerips.csv', 'r') as ipf:
    ips = ipf.readlines()
ips = [ip.strip() for ip in ips] # remove newlines

In [28]:
ips[0:5]

['61.240.144.66',
 '5.9.253.163',
 '65.242.160.99',
 '61.153.106.24',
 '::ffff:185.40.4.65']

In [31]:
ipset = set(ips) # remove duplicates

In [30]:
len(ipset)

4006

In [34]:
with open('../data/ip_geos.csv', 'r') as ipf:
    geod = ipf.readlines()
gathered = [g.split(',')[0] for g in geod]
gathered = set(gathered)
gathered

{'23.116.185.114', '::ffff:185.40.4.65'}

In [35]:
need_geos = ipset.difference(gathered)

In [36]:
len(need_geos)

4005

In [50]:
isinstance(need_geos, set)

True

Ok going to write a class, GeolocateIPs, that will do this for me.

In [58]:
3.236 % 1

0.2360000000000002

In [52]:
isinstance('aha', list)

False

## Test GeolocateIPs class

In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
scriptpath = '..'
sys.path.append(os.path.abspath(scriptpath))
from geolocate_ips import GeolocateIPs

In [2]:
geo = GeolocateIPs()

In [3]:
ips = geo.load_ip_list('../data/honeypot_10ksamp_attackerips.csv')

Found 4006 unique IP addresses.


In [4]:
len(ips)

4006

In [6]:
#geo.geo_ips(ips, '../data/ip_geos.csv')

In [72]:
import time
time.sleep(3)

In [78]:
300 % 300

0

In [83]:
import datetime

In [85]:
print(datetime.datetime.now())

2019-11-11 11:26:41.994817


In [92]:
print('{}: Reached hourly query limit. Sleeping for {} seconds.'.\
                    format(datetime.datetime.now(), 3))

2019-11-11 11:29:15.330493: Reached hourly query limit. Sleeping for 3 seconds.


Wrote it, now test it..

In [13]:
%%time
geo.geo_ips(list(ips)[:200], '../data/ip_geos.csv')

Previously geolocated 100 IP addresses.
Fetching 100 new geolocations. This will take at least 0.0 hours.
2019-11-11 11:49:23.224291: Writing IP geolocations for IPs 0-100 to file.
CPU times: user 2.38 s, sys: 141 ms, total: 2.53 s
Wall time: 29.3 s


Let's do them all!!

In [ ]:
%%time
geo.geo_ips(ips, '../data/ip_geos.csv')

Previously geolocated 200 IP addresses.
Fetching 3806 new geolocations. This will take at least 0.3 hours.
2019-11-11 11:58:07.173089: Return code: 404 for IP "". Adding IP to `failed_ips` list.
